## CLM-Demeter integration tutorial


#### This tutorial describes the steps necessary to produce CLM-derived input files for Demeter and execute Demeter using PIC using the created inputs.


#### The files stored to execute the CLM-Demeter experiment can be found here:
`/pic/projects/GCAM/demeter/runs/clm`

#### The following are the associated file paths and additional info to run the code:


In [6]:
import os

project_dir = '/pic/projects/GCAM/demeter/runs/clm'

# file created to spatial represent the base year of 2005 at 0.05 degree for each CLM PFT in units squared-degrees
original_baselayer_file = os.path.join(project_dir, 'inputs', 'observed', 'baselayerdata.csv')

# output file to save the base layer reclassified from CLM to Demeter land classes to
reclassified_baselayer_file = os.path.join(project_dir, 'inputs', 'observed', 'clm_baselayer_by_final_lcs.csv')

# file allocating CLM PFTs to Demeter's final land classes
spatial_allocation_file = os.path.join(project_dir, 'inputs', 'allocation', 'spatial_allocation_rules.csv')

# file allocating GCAM land classes to Demeter's final land classes
projected_allocation_file = os.path.join(project_dir, 'inputs', 'projected', 'gcam_allocation_rules.csv')

# output allocation file representing reclassification of GCAM land classes to Demeter's land classes
out_proj_allocation_file = os.path.join(project_dir, 'inputs', 'allocation', 'gcam_allocation_rules_reclass.csv')

# output allocation file representing reclassification of CLM PFTs to Demeter's land classes
out_spat_allocation_file = os.path.join(project_dir, 'inputs', 'allocation', 'spatial_allocation_rules_reclass.csv')

# file for land projections from GCAM for all years, subregions, and land classes
projected_file = os.path.join(project_dir, 'inputs', 'projected', 'gcam_ssp4_34_lumip.csv')

# output file to save the disaggregated land projections from GCAM to
out_projected_file = os.path.join(project_dir, 'inputs', 'projected', 'gcam_ssp4_34_lumip_disagg.csv')

# Either 'aez_id' or 'basin_id' depending on the version of GCAM used
metric = 'aez_id'

# list of GCAM years to process
gcam_year_list = [1990] + list(range(2005, 2105, 5))


#### Import the modules we need from this package:


In [7]:
import clm_demeter.reclassify_projected as rp
import clm_demeter.reclassify_allocation as ra

from clm_demeter.reclassify_base import ReclassBaselayer


### TODO:  Min to insert:
 -  the logic and any code that was used to create the original CLM base layer
 -  what CLM version and assumptions were used to generate the source data
 -  document how the transitions were mapped
 -  document how the treatment order was determined


### Step 1: Reclassify the existing CLM base layer that represents CLM PFTs to Demeter's desired output land classes


#### Check out help on our `ReclassBaselayer` class to see what it requires:

In [18]:
help(ReclassBaselayer)


Help on class ReclassBaselayer in module clm_demeter.reclassify_base:

class ReclassBaselayer(builtins.object)
 |  Reclassify CLM PFTs in the baselayer for Demeter to Demeter final landcover classes.
 |  
 |  :param clm_baselayer_file:              Full path with file name and extension to
 |                                          the CLM baselayer prepared for Demeter
 |                                          in units square-degrees.
 |  
 |  :param clm_spatial_allocation_file:     Full path with file name and extension to
 |                                          the Demeter spatial allocation file for CLM
 |                                          landclasses.
 |  
 |  :param out_clm_baselayer_file:          Full path with file name and extension to
 |                                          to save the reclassified baselayer.
 |  
 |  :method demeter_to_clm_map:             Create a dictionary of Demeter final landclasses
 |                                          to CLM P

#### Generate a reclassififed base layer for Demeter from the original CLM base layer:


In [24]:
base = ReclassBaselayer(original_baselayer_file, spatial_allocation_file, reclassified_baselayer_file)


### Step 2: Disaggregate any projected land class areas per subregion by what is fractionally represented in the base layer.  This prevents Demeter's built-in method from spliting the area evenly regardless of what is in the observed data.


#### Check out the help on the batch function as well as the class that does the work:


In [3]:
help(rp.batch_process_split)


Help on function batch_process_split in module clm_demeter.reclassify_projected:

batch_process_split(projected_allocation_file, observed_baselayer_file, projected_file, out_projected_file, metric, gcam_year_list)
    Batch process projected land class disaggregation.
    
    :param projected_allocation_file:   Full path with file name and extension of the projected allocation file from
                                        Demeter that maps GCAM land classes to the final land cover classes in
                                        Demeter.
    
    :param observed_file:               Full path with file name and extension of the observed data file to be used in the
                                        Demeter run.
    
    :param projected_file:              Full path with file name and extension of the projected data file that has been
                                        extracted from a GCAM output database for use with Demeter.
    
    :param out_projected_file:        

In [4]:
help(rp.GcamLandclassSplit)


Help on class GcamLandclassSplit in module clm_demeter.reclassify_projected:

class GcamLandclassSplit(builtins.object)
 |  Split a GCAM landclass into multiple classes based on the fractional amount present in the observed data per
 |  subregion.  This method is more desirable than the default "even percentage" split that Demeter conducts.  The
 |  output file replaces the GCAM target landclass (e.g. RockIceDesert) with the user-selected classes (e.g. snow and
 |  sparse) per subregion.  The new file becomes what is referenced as the projected file in Demeter.
 |  
 |  :param observed_file:               Full path with file name and extension of the observed data file to be used in the
 |                                      Demeter run.
 |  
 |  :param projected_file:              Full path with file name and extension of the projected data file that has been
 |                                      extracted from a GCAM output database for use with Demeter.
 |  
 |  :param target_lan

#### Run batch disaggregation of land classes:


In [4]:
proj = rp.batch_process_split(projected_allocation_file, 
                                reclassified_baselayer_file,
                                projected_file,
                                out_projected_file,
                                metric,
                                gcam_year_list)


Disaggregating projected land class 'biomass' to '['Bioenergy_rf', 'Bioenergy_irr']'
Disaggregating projected land class 'Corn' to '['Corn_rf', 'Corn_irr']'
Disaggregating projected land class 'FiberCrop' to '['Cotton_rf', 'Cotton_irr']'
Disaggregating projected land class 'FodderGrass' to '['Fodder_rf', 'Fodder_irr']'
Disaggregating projected land class 'FodderHerb' to '['Fodder_rf', 'Fodder_irr']'
Disaggregating projected land class 'Forest' to '['NET_tem', 'NET_bor', 'NDT_bor', 'BET_tro', 'BET_tem', 'BDT_tro', 'BDT_tem', 'BDT_bor']'
Disaggregating projected land class 'Grassland' to '['C3_gra_arc', 'C3_gra', 'C4_gra']'
Disaggregating projected land class 'MiscCrop' to '['OtherCrop_rf', 'OtherCrop_irr']'
Disaggregating projected land class 'OilCrop' to '['Soy_rf', 'Soy_irr']'
Disaggregating projected land class 'OtherArableLand' to '['OtherCrop_rf', 'OtherCrop_irr']'
Disaggregating projected land class 'OtherGrain' to '['OtherCrop_rf', 'OtherCrop_irr']'
Disaggregating projected land 

### Step 3:  Reclassify the spatial and projected allocation file


#### Check out help on the two functions we need:


In [9]:
help(ra.reclass_projected_allocation)


Help on function reclass_projected_allocation in module clm_demeter.reclassify_allocation:

reclass_projected_allocation(projected_data_file, projected_allocation_file, out_proj_allocation_file)
    Reclassify the projected allocation file for Demeter to account for classes that have
    been disaggregated to Demeter's final land classes.
    
    :param projected_file:              Full path with file name and extension of the projected data file that has been
                                        extracted from a GCAM output database for use with Demeter.
    
    :param projected_allocation_file:   Full path with file name and extension of the projected allocation file from
                                        Demeter that maps GCAM land classes to the final land cover classes in
                                        Demeter.
    
    :param out_proj_allocation_file:    Full path with file name and extension for the reclassified projected allocation
                          

In [10]:
help(ra.reclass_spatial_allocation)


Help on function reclass_spatial_allocation in module clm_demeter.reclassify_allocation:

reclass_spatial_allocation(spatial_allocation_file, out_spatial_allocation_file)
    Reclassify the spatial allocation file for Demeter to 1:1 land class relationships
    for Demeter. Using the output of this functions assumes that the user has already
    created a reclassified base layer.
    
    :param spatial_allocation_file:     Full path with file name and extension to
                                        the Demeter spatial allocation file for CLM
                                        landclasses.
    
    :param out_spat_allocation_file:    Full path with file name and extension for the reclassified spatial allocation
                                        file.



#### Reclass the projected allocation file:


In [14]:
ra.reclass_projected_allocation(out_projected_file, projected_allocation_file, out_proj_allocation_file)


Projected land class 'Jatropha' not in Demeter land classes.


#### Reclass the spatial allocation file:


In [16]:
ra.reclass_spatial_allocation(spatial_allocation_file, out_spat_allocation_file)


### Step 4:  Set-up Demeter run


